In [1]:
from mqtt import Light_Switch
from IPython.display import display
import ipywidgets as widgets
import time, threading
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.colors as mc
import datetime
import pandas as pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import glob
import base64
from IPython.display import HTML



In [2]:
def map_values(x, in_min, in_max, out_min, out_max):
    return (x-in_min) * (out_max - out_min) / (in_max - in_min) + out_min

def rle(inarray):
        """ run length encoding. Partial credit to R rle function. 
            Multi datatype arrays catered for including non Numpy
            returns: tuple (runlengths, startpositions, values) """
        ia = np.asarray(inarray)                  # force numpy
        n = len(ia)
        if n == 0: 
            return (None, None, None)
        else:
            y = np.array(ia[1:] != ia[:-1])     # pairwise unequal (string safe)
            i = np.append(np.where(y), n - 1)   # must include last element posi
            z = np.diff(np.append(-1, i))       # run lengths
            p = np.cumsum(np.append(0, z))[:-1] # positions
            return(z, p, ia[i])

In [36]:
help(Light_Switch)

Help on class Light_Switch in module mqtt:

class Light_Switch(MQTT_connection)
 |  A class that represents a thread of control.
 |  
 |  This class can be safely subclassed in a limited fashion. There are two ways
 |  to specify the activity: by passing a callable object to the constructor, or
 |  by overriding the run() method in a subclass.
 |  
 |  Method resolution order:
 |      Light_Switch
 |      MQTT_connection
 |      threading.Thread
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, *args, **kwargs)
 |      This constructor should always be called with keyword arguments. Arguments are:
 |      
 |      *group* should be None; reserved for future extension when a ThreadGroup
 |      class is implemented.
 |      
 |      *target* is the callable object to be invoked by the run()
 |      method. Defaults to None, meaning nothing is called.
 |      
 |      *name* is the thread name. By default, a unique name is constructed of
 |      the form "Thr

In [34]:

light_switch_button = widgets.Button(description = "Switch Light", button_style = "info")
light_switch = Light_Switch(broker_address = "192.168.1.3", client_name = "switch")
def switch_light(event):
    text.value = f"Switched at {time.strftime('%H:%M:%S')}"
    light_switch.switch()
light_switch_button.on_click(switch_light)

text = widgets.Textarea()
display(widgets.HBox(children = [text, light_switch_button]))

class Monitor:
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        self.ax.xaxis_date()
        self.make_ui()
        self.update_plot("mock_event")
        self.alive = False
#         self.start_update_thread("mock_event")
    def start_update_thread(self, event):
        if self.alive:
            self.alive = False
        else:
            self.t = threading.Thread(target = self.update_thread, args = ())
            self.t.start()
        
            
    def update_thread(self):
        self.alive = True
        counter = 0
        icon = ["|","--"]
        last_time = time.strftime("%H:%M:%S")
        while self.alive:
            interval = self.update_interval.value
            time.sleep(1)
            counter += 1
            self.ax.set_title(f"{icon[counter%2]} last update {last_time} {icon[counter%2]}")
            self.fig.canvas.draw()
            if counter % interval == 0:
                self.update_plot("mock_event")
                last_time = time.strftime("%H:%M:%S")
                
    def make_ui(self):
        self.file_dropdown = widgets.Dropdown(options = glob.glob("/mnt/NAS/optoworld_logs/*.txt"), 
                                              value = glob.glob("/mnt/NAS/optoworld_logs/*.txt")[-1])
        self.run_button = widgets.Button(description = "Update Plot")
        self.run_button.on_click(self.update_plot)
        self.download_button = widgets.Button(description = "Make Download Link")
        self.download_button.on_click(self.show_download_link)
        self.hbox = widgets.HBox(children = [self.file_dropdown, self.run_button, 
                                             self.download_button])
        
        self.start_auto_update = widgets.Button(description = "Start/Stop Autoupdate")
        self.start_auto_update.on_click(self.start_update_thread)
        self.update_interval = widgets.IntSlider(description = "Update Interval", min = 5, max = 300, 
                                                 value = 10, continuous_update = False)
        self.hbox2 = widgets.HBox(children = [self.start_auto_update, self.update_interval])
        self.vbox = widgets.VBox(children = [self.hbox, self.hbox2])
        display(self.vbox)
        
    
    def create_download_link(self,f, title = "Download CSV file", filename = "data.csv"):  
        filename = f
        title = f"Download file:  {filename}"
        filename = f.split("/")[-1]
        csv = open(f,'r').read()
        b64 = base64.b64encode(csv.encode())
        payload = b64.decode()
        html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
        html = html.format(payload=payload,title=title,filename=filename)
        return HTML(html)

    def show_download_link(self, event):
        display(self.create_download_link(self.file_dropdown.value))
        
    def update_plot(self, event):
        
        file = pd.read_csv(self.file_dropdown.value, sep = "\t")

        min_temp = file["temperature"].min()
        max_temp = file["temperature"].max()
        lengths, starts, values =rle(file["lightstate"])
        self.ax.clear()
        for length, start, value in zip(lengths, starts, values):
            if value == 1:
    #             print(length, start)
                x = mdates.date2num(pd.to_datetime(file["time"].iloc[start]))
                y = min_temp -1
                try:
                    l = mdates.date2num(pd.to_datetime(file["time"].iloc[start+length])) - x
                except:
                    l = mdates.date2num(pd.to_datetime(file["time"].iloc[-1])) - x
                h = max_temp - min_temp + 2
                rect = plt.Rectangle((x,y), l, h, facecolor = "lightblue", alpha = 0.3)
                self.ax.add_artist(rect)    
        self.ax.plot(pd.to_datetime(file["time"]), file["temperature"], c = "gray", alpha = 0.1, lw = 0.5)
        norm = mc.Normalize(vmin = 20, vmax = 30)
        self.ax.scatter(pd.to_datetime(file["time"]), file["temperature"], c = norm(file["temperature"].values), cmap = "RdBu_r", s = 1)
#         self.fig.autofmt_xdate()
        

In [35]:
%matplotlib widget
Monitor();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …